In [2]:
# Load up full Phoenix dataset - drop all non-relevant categories
import pickle
import psycopg2
import pandas as pd
# Extract the data we're interested in
conn = psycopg2.connect(database='Yelp', user='brooks', password='davidson', host='localhost')

sql_query = f"""
SELECT * FROM t_phoenix WHERE ( NOT (categories LIKE '%Restaurant%' OR categories LIKE '%Bar%' 
OR categories LIKE '%Festivals%' OR categories LIKE '%Hair Salons%'
OR categories LIKE '%Food Truck%' OR categories LIKE '%Hotel%'
OR categories LIKE '%Contractor%' OR categories LIKE '%Apartment%'
OR categories LIKE '%Cater%' OR categories LIKE '%Local Flavor%')
OR (categories LIKE '%Grocery%') );
"""
phx_from_sql = pd.read_sql_query(sql_query,conn)
phx_from_sql.set_index('business_id',inplace=True)
#pickle.dump( phx_from_sql, open( "phx_db.p", "wb" ) )


In [3]:
# Parse saved HTML pages - try to identy top link (drop anything that comes from an aggregator)
import numpy as np
from bs4 import BeautifulSoup

all_ids = phx_from_sql.index.values
all_urls = np.zeros(len(all_ids),dtype=np.object)
path1 = './google_search/'
collectors = ['yelp', 'facebook', 'mapquest', 'google', 'yahoo', 'bing', 'duckduckgo', 'groupon','yellowpages','tripadvisor']

for i,id_tmp in enumerate(all_ids):
    try:
        file = open(path1+id_tmp+'.html','r') 
        contents = file.read()
        soup = BeautifulSoup(contents,'html.parser')
        all_links = soup.find_all('a',class_='result__check')
        j = -1
        while (all_urls[i]==0) and (j <(len(all_links)-1)):
            j += 1
            tmp = all_links[j]['href']
            if not any(x in tmp for x in collectors):
                all_urls[i] = tmp
    except:
        print('skipping', id_tmp)




In [4]:
# Concatenate list of business IDs and URLs - save as dataframe
url_df = pd.DataFrame(np.concatenate((all_ids[np.newaxis].T,all_urls[np.newaxis].T),axis=1)
                      ,columns=['business_id','url'])

pickle.dump(url_df, open( "df_URLs.p", "wb" ) )


In [97]:
for x in b_link[0:100]:
    print(x)


https://www.circlek.com/
https://www.walmart.com/store/2113/phoenix-az/details
https://www.yellowpages.com/scottsdale-az/mip/john-yourk-jewelry-designs-6856626
https://www.tileshop.com/store-locator/phoenix.do
https://www.placelookup.net/apples-pet-boutique_826975
https://www.wolfevalleyelectric.com/
https://www.azinternationalmarketplace.com/
http://www.phoenixmetro.com/cities/tempe/stinkys_xpress_lube.html
https://www.fifthestatetattoo.com/
https://www.circlek.com/
https://www.yogabreeze.com/
https://www.zillow.com/profile/EvanandJentriYoung/
https://locations.einsteinbros.com/us/az/phoenix
http://metrorefreshdentistry.com/
https://www.yellowpages.com/phoenix-az/deer-valley-mechanical
https://www.redcanyondentistry.com/
http://azsunchiro.com/
https://petdentalusa.com/
https://antiquitieswarehouse.com/
https://www.forlocations.com/freshandeasy-Phoenix-hours-locations/AZ
https://www.harborfreight.com/
https://centerforhealingaz.com/
https://www.indexusedcars.com/dealers/nick-mayer-hyu

In [109]:
phx_from_sql = pickle.load(open( "phx_db.p", "rb" ) )

In [110]:
phx_from_sql.head(5)

,name,address,city,latitude,longitude,stars,categories
business_id,,,,,,,
bFzdJJ3wp3PZssNEsyU23g,Geico Insurance,211 W Monroe St,Phoenix,33.449999,-112.076979,1.5,"Insurance, Financial Services"
45bWSZtniwPRiqlivpS8Og,The Coffee Bean & Tea Leaf,"20235 N Cave Creek Rd, Ste 1115",Phoenix,33.671375,-112.030017,4.0,"Coffee & Tea, Food"
guRyrKMBDkcHExGzwxSfvg,"ADT Security Services, Inc.",,Phoenix,33.431665,-112.102959,1.0,"Home Services, Home Automation, Security Systems"
iTZDPTc36guXhNhoK-jwmw,Wellsprings of Gilbert,3319 S Mercy Rd,Gilbert,33.290596,-111.747814,5.0,"Health & Medical, Physical Therapy, Retirement..."
Zkesf4VRamyOBswJA4_ZhQ,Audio Express,"2929 North 75th Ave, Suite 26",Phoenix,33.509413,-112.203724,3.5,"Automotive, Auto Customization, Shopping, Auto..."


NameError: name 'b_link' is not defined